In [11]:
from IPython.display import display
import spotipy
import spotipy.util as util


client_id = 'b69a9985fa8842deb0691b2d0e3f0b69'
client_secret = 'd9e9ae2924174c139a5a9ccb303f9f3a'
redirect_uri = 'http://localhost/'

username = '22mrmbu7oumkrb56tcsclawdi'

token = util.prompt_for_user_token(username, 'user-library-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)
tracks = []

offset = 0
while True:
    track_set = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
    if track_set:
        tracks += track_set
    else:
        break

    offset += 50
print("Tracks fetched")

artist_tracks = {}
all_artists = set()
artist_related = {}
for i in range(len(tracks)):
    tracks[i] = tracks[i]['track']
    track_id = tracks[i]['id']
    artist_id = tracks[i]['artists'][0]['id']
    
    if artist_id not in artist_tracks:
        artist_tracks[artist_id] = [track_id]
    else:
        artist_tracks[artist_id].append(track_id)
    
    for artist in tracks[i]['artists']:
        all_artists.add(artist['id'])
        
i = 0
for artist_id in all_artists:
    if artist_id not in artist_related:
        related_list = sp.artist_related_artists(artist_id)['artists']
        artist_related[artist_id] = []
        
        for related in related_list:
            related_id = related['id']
            artist_related[artist_id].append(related_id)
            
            if related_id not in artist_related:
                sub_related_list = sp.artist_related_artists(related_id)['artists']
                artist_related[related_id] = []
                for sub_related in sub_related_list:
                    artist_related[related_id].append(sub_related['id'])
                    
    i += 1
    if i % 500 ==0:
        print("%d/%d artists processed" % (i, len(all_artists)))
                
print("Artists fetched")

Tracks fetched
0/3781 tracks processed
500/3781 tracks processed
1000/3781 tracks processed
1500/3781 tracks processed
retrying ...1secs
2000/3781 tracks processed
2500/3781 tracks processed
3000/3781 tracks processed
3500/3781 tracks processed
Artists fetched


In [12]:
import networkx as nx

def chunks(n, x):
    return [x[i:i + n] for i in range(0, len(x), n)]

artist_genres = {}
for chunk in chunks(50, list(artist_tracks.keys())):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']
print("Genres fetched")
        
G = nx.Graph()
for artist in artist_related:
    for related_artist in artist_related[artist]:
        G.add_edge(artist, related_artist)

Genres fetched


In [14]:
nx.write_graphml(G, 'related_artists.graphml')

In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

G = nx.Graph()
G.add_edge(1,2)
nodes = list(G.nodes())
paths = list(nx.all_pairs_shortest_path_length(G))
distance = np.array(np.zeros(len(nodes), len(nodes)))
for i in range(len(nodes)):
    if paths[i][0] != nodes[i]:
            print("Not same order as nodeslist!")
    for j in range(len(nodes)):
        distance[i][j] = paths[i][1][nodes[j]]

similarity = np.exp(-distance / distance.std())

In [ ]:
np.savetxt('distance.txt', distance)

In [ ]:
import sklearn.cluster

clustering = sklearn.cluster.SpectralClustering(n_clusters=5, affinity='precomputed').fit_predict(similarity)

artist_cluster = {}
cluster_artists = {}
nodes = list(G.nodes())
for i in range(len(nodes)):
    artist_cluster[nodes[i]] = clustering[i]
    cluster_artists[clustering[i]] = nodes[i]

In [ ]:
counts = {'cluster': [], 'genre': [], 'count': []}
for cluster in cluster_artists:
    genre_count = dict()
    artist_count = 0
    for artist in cluster_artists[cluster]:
        if artist in artist_tracks:
            artist_count += 1
            for genre in artist_genres[artist]:
                if genre in genre_count:
                    genre_count[genre] += 1
                else:
                    genre_count[genre] = 1
            for genre, count in genre_count.items():
                counts['cluster'].append(cluster)
                counts['genre'].append(genre)
                counts['count'].append(count / artist_count)

counts = pd.DataFrame(counts)
counts.sort_values('count', ascending=False, inplace=True)
original_counts = counts.copy()

clusters_to_name = set(counts['cluster'].unique())
playlists = dict()
while clusters_to_name:
    cluster = counts.iloc[0]['cluster']
    name = counts.iloc[0]['genre']
    playlists[name] = cluster
    counts = counts[ (counts['cluster']!=cluster) & (counts['genre']!=name) ]
    clusters_to_name.remove(cluster)
    
playlist_tracks = dict()
from collections import OrderedDict
for name in playlists:
    playlist_tracks[name] = []
    cluster = playlists[name]
    for artist in cluster_artists[cluster]:
        playlist_tracks[name] += artist_tracks[artist]
    

In [ ]:
token = util.prompt_for_user_token(username, 'playlist-modify-private', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

for name in playlist_tracks:
    id = sp.user_playlist_create(username, 'sortify '+name, public=False)['id']
    for chunk in chunks(100, playlist_tracks[name]):
        sp.user_playlist_add_tracks(username, id, chunk)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(playlists)